In [2]:
import glob
import numpy as np 
import pandas as pd

# 讀入資料

In [5]:
data=pd.read_csv('/Users/chenjessie/Desktop/全部指標＿predict1-5.csv',index_col=0)
data

,新屋銷售,新屋開工,成屋銷售,CPI,coreCPI,個人支出,信心,失業率,GDP,predict
日期,,,,,,,,,,
2019-12-01,71.9,136.5,535,0.3,0.2,0.4,99.300000,3.5,2.10000,-5
2019-11-01,73.3,131.4,546,0.4,0.2,0.3,96.800000,3.6,2.10000,0
2019-10-01,70.1,125.6,538,0.0,0.1,0.2,95.500000,3.5,1.90000,2
2019-09-01,71.3,136.4,549,0.1,0.3,0.1,93.200000,3.7,2.00000,3
2019-08-01,63.5,119.1,542,0.3,0.3,0.6,89.800000,3.7,2.00000,2
...,...,...,...,...,...,...,...,...,...,...
1970-05-01,44.5,126.0,151,0.5,0.7,0.6,85.917625,4.6,-0.25408,5
1970-04-01,38.9,132.0,137,0.5,0.8,-0.3,85.917625,4.4,-0.25408,-5
1970-03-01,37.3,131.0,146,0.5,0.5,0.8,78.100000,4.2,-0.25408,-5


# 統計

In [8]:
data.groupby(['predict'],as_index=False)['predict'].agg({'cnt':'count'})

,predict,cnt
0,-5,57
1,-4,12
2,-3,30
3,-2,38
4,-1,51
5,0,115
6,1,75
7,2,48
8,3,58
9,4,37


In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data[['新屋銷售','新屋開工','成屋銷售','CPI','coreCPI','個人支出','信心','失業率','GDP']],data[['predict']],test_size=0.2,random_state=9)

# 標準化x值

In [31]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

# SVM
- sklearn.svm.SVC(C=1.0, kernel=’rbf’, degree=3, gamma=0.0, coef0=0.0, shrinking=True, probability=False,tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=None)

## 1. Sigmoid

In [41]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svm = SVC(kernel='sigmoid', probability = True,gamma='auto')  #linear/rbf/poly
svm.fit(x_train_std,y_train['predict'].values)
x_test_result = svm.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test, x_test_result))

Accuracy: 0.225


In [36]:
x_test_result

array([ 5,  0,  1,  1,  5,  5, -5,  5,  0, -5,  0,  0,  0,  5,  0,  5,  0,
        1,  5,  0,  5,  0,  0,  5,  0,  0, -5,  5,  5,  0,  0,  0,  0,  5,
        0,  5,  0, -5,  5,  0,  0,  0,  0,  0,  0, -5,  0,  0,  0,  0,  5,
        5,  0,  1,  5,  0,  0,  0,  0,  0,  5,  0,  0, -5,  1,  0,  1,  0,
        1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  5,
        1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        5,  0,  0,  0,  5,  0,  0,  0,  1,  0,  1,  0,  0,  5,  0,  0,  0,
       -5])

## 2. poly


In [11]:
from sklearn.svm import SVC
svm = SVC(kernel='poly', probability = True)  #linear/rbf/poly
svm.fit(x_train_std,y_train['predict'].values)
x_test_result1 = svm.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test, x_test_result1))

Accuracy: 0.18333333333333332


## 3. rbf

In [12]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', probability = True)  #linear/rbf/poly
svm.fit(x_train_std,y_train['predict'].values)
x_test_result2 = svm.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test, x_test_result2))

Accuracy: 0.2111111111111111


# Decision Tree
- DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,max_features=None, max_leaf_nodes=None,min_impurity_split=1e-07, min_samples_leaf=1,min_samples_split=2, min_weight_fraction_leaf=0.0,presort=False, random_state=None, splitter='best')

In [13]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'gini',max_depth=2) 
tree.fit(x_train_std,y_train)
x_test_result3=tree.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test,x_test_result3))

Accuracy: 0.17777777777777778


# Random Forest

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
model = BaggingClassifier(tree, n_estimators=60, max_samples=0.8,)

model.fit(x_train_std,y_train)
x_test_result4=model.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test,x_test_result4))

Accuracy: 0.18333333333333332


/opt/anaconda3/envs/env1/lib/python3.7/site-packages/sklearn/ensemble/_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Conclusion
    SVM 的 kernel＝“linear”,"rbf"
    Decision tree 的max_depth = 2
    的準確度為最高，皆為 86.18%